In [1]:
#Import dependencies:

import requests
import pandas as pd

In [6]:
#Define zipcode lookup function:

def zip_lookup(dataframe):
    info_dict = {}
    base_url = 'https://us-street.api.smartystreets.com/street-address?'
    aid = 'd7b9e612-80b0-18b4-cc6f-d1116195180c'
    token = 'nkkpYkl7heqS3sVZRNuX'
    zipcodes = []
    for row in dataframe.itertuples():
        street = row.Address
        city = row.City
        state = row.State
        parameters = {'auth-id': aid, 'auth-token':token, 'match': 'match', 'street': street, 'city': city, 'state': state}
        response =  requests.get(base_url, params=parameters)
        results = response.json()
        try:
            zipcode = results[0]['components']['zipcode']
        except:
            zipcode = 'None'
        zipcodes.append(zipcode)
    return(zipcodes)

In [12]:
connected = pd.read_csv('../Phone API Work/Ekata/ekata_connected.csv')
disconnected = pd.read_csv('../Phone API Work/Ekata/ekata_disconnected.csv')

In [11]:
#Read csvs

connected_df = pd.read_csv('combination_connected.csv')
disconnected_df = pd.read_csv('combination_disconnected.csv')

FileNotFoundError: [Errno 2] File b'combination_connected.csv' does not exist: b'combination_connected.csv'

In [7]:
#Lookup zips

dis_zip = zip_lookup(disconnected)
con_zip = zip_lookup(connected)

In [13]:
connected['Zipcode'] = con_zip

In [14]:
disconnected['Zipcode'] = dis_zip

In [42]:
#Read combo csv:

combo = pd.read_csv("combination_connected2.csv")
combo.head()

,Unnamed: 0,Address_GetPhoneInfo,City_GetPhoneInfo,Date_GetPhoneInfo,Name_GetPhoneInfo,Notes,OFFICE_TELEPHONE,PhoneType_GetPhoneInfo,Provider,QualityScore,...,Error,Industry,LineType,Name_Ekata,PhoneType_Ekata,State_Ekata,Valid,Address_Ekata,OFFICE_ADDRESS_LINE_2,Zipcode_Ekata
0,0,2755 S HIGHWAY 14 STE 2500,GREER,12/23/2008,MEDICAL GROUP OF THE CAROLINASDIVISION OF,"IsMailable,IsConnected,IsPorted",8648499555,BUSINESS,NUVOX COMMUNICATIONS,HIGH,...,None,Ambulatory Health Care Services,Landline,"Cash, Allyson L N.P.",Business,SC,True,2755 S Highway 14 Ste 2200,2755 S HIGHWAY 14 STE 2200,29650
1,1,1501 HOUSTON ST,CASTROVILLE,10/12/2017,LITTLE ALSACE URGENT CARE CENTER CHILD CARE,"IsMailable,IsConnected,IsPorted",8305383550,BUSINESS,BANDWIDTH.COM - TX,HIGH,...,None,Ambulatory Health Care Services,NonFixedVOIP,"Fritz, Rebecca PA",Business,TX,True,1501 Houston St,1501 HOUSTON ST,78009
2,2,NaN,BURLINGTON,NaN,BRUCE TRONIC,NaN,7817448046,RESIDENTIAL,VERIZON NEW ENGLAND,LOW,...,None,Ambulatory Health Care Services,Landline,"Bilic, Masha MD",Business,MA,True,41 Mall Rd,41 MALL RD,1805
3,3,475 IRVING AVE STE 210,SYRACUSE,5/28/2015,MADISON IRVING PEDIATRICS DEVELOPMENTAL HEALTH,"IsMailable,IsConnected,IsPorted",3154712646,BUSINESS,NORTHLAND NETWORKS,HIGH,...,None,Ambulatory Health Care Services,Landline,Madison Irving Pediatrics Pc,Business,NY,True,475 Irving Ave Ste 210,475 IRVING AVE STE 210,13210
4,4,160 OVERLOOK AVE STE 1A,HACKENSACK,1/25/2018,ANDREW FINK MD PHYSICIANS SURGEONS,"IsMailable,IsConnected,IsPorted",2014883131,BUSINESS,"AIRUS, INC. - NJ",HIGH,...,None,Ambulatory Health Care Services,NonFixedVOIP,"Fink, Andrew MD",Business,NJ,True,385 Prospect Ave Ste 20,385 PROSPECT AVE STE 200,7601


In [47]:
#Check for zipcode matches

ekata_match = 0
get_match = 0
api_match = 0
for row in combo.itertuples():
    if str(row.Zipcode_Ekata) == str(row.OFFICE_ADDRESS_ZIP):
        ekata_match += 1
    if row.Zipcode_GetPhoneInfo== row.OFFICE_ADDRESS_ZIP:
        get_match += 1
    if str(row.Zipcode_Ekata) == str(row.Zipcode_GetPhoneInfo):
        api_match += 1
    else:
        print(F'{row.Zipcode_Ekata} {row.Zipcode_GetPhoneInfo} {row.OFFICE_ADDRESS_ZIP}')
print (f'{ekata_match} zipcodes match between Ekata and WSLive \n{get_match} zipcodes match between GetPhoneInfo and WSLive \n{api_match} zipcodes match between Ekata and GetPhoneInfo')

1805 1803 1805
89144 89074 89193
 None 85283 85283
11214 11235 11214
7103 7107 7101
40207 40241 40241
35 zipcodes match between Ekata and WSLive 
35 zipcodes match between GetPhoneInfo and WSLive 
34 zipcodes match between Ekata and GetPhoneInfo


In [15]:
connected.head()

,Address,AlternatePhone,Carrier,City,Commercial,Date,Error,Industry,LineType,Name,OFFICE_TELEPHONE,PhoneType,State,Valid,Zipcode
0,385 Prospect Ave Ste 20,None,Onvoy/3,Hackensack,True,2/28/2019,None,Ambulatory Health Care Services,NonFixedVOIP,"Fink, Andrew MD",2014883131,Business,NJ,True,07601
1,23077 Greenfield Rd Ste 195,None,Comcast,Southfield,True,11/15/2016,None,Ambulatory Health Care Services,FixedVOIP,Stanley Keemer Saunders MD,2484230700,Business,MI,True,48075
2,475 Irving Ave Ste 210,18003681019,Northland Networks Ltd,Syracuse,True,2/28/2019,None,Ambulatory Health Care Services,Landline,Madison Irving Pediatrics Pc,3154712646,Business,NY,True,13210
3,613 E Bloomington St # 100,None,CenturyLink,Iowa City,True,11/18/2016,None,Ambulatory Health Care Services,Landline,Cancer Care Of Iowa City,3193393917,Business,IA,True,52245
4,1037 Main St,None,Blackfoot Communications,Corvallis,True,5/6/2017,None,Ambulatory Health Care Services,Landline,April Lynn Weinberger MD,4069614661,Business,MT,True,59828


In [18]:
connected.to_csv('../Phone API Work/Ekata/ekata_connected.csv')
disconnected.to_csv('../Phone API Work/Ekata/ekata_disconnected.csv')

In [17]:
connected

,Address,AlternatePhone,Carrier,City,Commercial,Date,Error,Industry,LineType,Name,OFFICE_TELEPHONE,PhoneType,State,Valid,Zipcode
0,385 Prospect Ave Ste 20,None,Onvoy/3,Hackensack,True,2/28/2019,None,Ambulatory Health Care Services,NonFixedVOIP,"Fink, Andrew MD",2014883131,Business,NJ,True,07601
1,23077 Greenfield Rd Ste 195,None,Comcast,Southfield,True,11/15/2016,None,Ambulatory Health Care Services,FixedVOIP,Stanley Keemer Saunders MD,2484230700,Business,MI,True,48075
2,475 Irving Ave Ste 210,18003681019,Northland Networks Ltd,Syracuse,True,2/28/2019,None,Ambulatory Health Care Services,Landline,Madison Irving Pediatrics Pc,3154712646,Business,NY,True,13210
3,613 E Bloomington St # 100,None,CenturyLink,Iowa City,True,11/18/2016,None,Ambulatory Health Care Services,Landline,Cancer Care Of Iowa City,3193393917,Business,IA,True,52245
4,1037 Main St,None,Blackfoot Communications,Corvallis,True,5/6/2017,None,Ambulatory Health Care Services,Landline,April Lynn Weinberger MD,4069614661,Business,MT,True,59828
5,121 Cahill Rd Ste 204,18002454362,CenturyLink,Branson,True,11/18/2016,None,Ambulatory Health Care Services,Landline,Branson Neurology & Pain Center,4173357222,Business,MO,True,65616
6,6780 Mayfield Rd,14403124500,Ameritech,Cleveland,True,2/28/2019,None,Ambulatory Health Care Services,Landline,Hillcrest Hospital,4403124569,Business,OH,True,44124
7,None,None,MCI Metro ATS,Phoenix,True,None,None,None,Landline,Cigna Corp,4803455200,Business,AZ,True,None
8,3999 Dutchmans Ln Ste 6f,15023945600,BellSouth,Louisville,True,5/6/2017,None,Ambulatory Health Care Services,Landline,Jennifer M Brey MD,5023945678,Business,KY,True,40207
9,275 Sandwich St,None,CTC Communications,Plymouth,True,2/28/2019,None,Ambulatory Health Care Services,Landline,"Byse, Barbara H MD",5087462000,Business,MA,True,02360


In [25]:
WRONG = pd.read_csv('../Phone API Work/Ekata/ekata_wrong_numbers.csv')

In [26]:
zip_lookup(WRONG)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [27]:
WRONG

,Address,AlternatePhone,Carrier,City,Commercial,Date,Error,Industry,LineType,Name,OFFICE_TELEPHONE,PhoneType,State,Valid
0,444 Foxon Rd,NaN,Level 3 Communications,East Haven,True,8/28/2018,NaN,Ambulatory Health Care Services,NonFixedVOIP,"Di Capua, Paul D MD",2035335911,Business,CT,True
1,453 John Lewis Freedom Pkwy NE,1.800550e+10,BellSouth,Atlanta,True,11/20/2016,NaN,Social Assistance,Landline,Carter Center,4044205100,Business,GA,True
2,3401 Ludington St,NaN,Ameritech,Escanaba,True,8/28/2018,NaN,Ambulatory Health Care Services,Landline,"Riley Sr, Alvin DO",9067863311,Business,MI,True
3,4770 Amoco Dr,NaN,Level 3 Communications,Moss Point,True,8/28/2018,NaN,Ambulatory Health Care Services,NonFixedVOIP,"Boykin, Ilene N.P.",2284749511,Business,MS,True
4,9179 W Tbird Rd Ste B105,NaN,Coxcom,Peoria,True,11/15/2016,NaN,Ambulatory Health Care Services,FixedVOIP,Amardeep Sodhi MD,6238159733,Business,AZ,True
5,13212 Shaker Sq Ste 204,NaN,ICG Telecom Group,Cleveland,True,11/17/2016,NaN,"Professional, Scientific, and Technical Services",NonFixedVOIP,Perspectus Architecture,2167521800,Business,OH,True
6,1000 Central St Ste 717,NaN,Onvoy/5,Evanston,True,5/6/2017,NaN,Ambulatory Health Care Services,NonFixedVOIP,Susan L Warner MD,8478641200,Business,IL,True
7,2144 Fairfax Ave,NaN,Teleport Communications America,Nashville,True,11/19/2016,NaN,Social Assistance,Landline,Ronald Mcdonald House,6153434000,Business,TN,True
8,410 W 10th St Ste 1100,NaN,Teleport Communications America,Indianapolis,True,5/6/2017,NaN,Ambulatory Health Care Services,Landline,Anantha Shekhar MD,3172789100,Business,IN,True
9,1201 Monument Rd Apt 200,1.904728e+10,Bandwidth SMSEnabled,Jacksonville,True,8/28/2018,NaN,Ambulatory Health Care Services,NonFixedVOIP,"Nimmo-Olsen, Whitney R",9047275151,Business,FL,True
